In [12]:
library(Seurat)
library(Matrix)
library(rhdf5)

In [2]:
data_path_ref='/home/khrovatin/retinal/data/counts/'
data_path='/home/khrovatin/retinal/data/human/'
data_path_shared_all='/share/LBI_share/public/retina/human\ snRNAseq\ data_10x_ChenLab_ZupanLab/humansn_10x_storage/'
data_path_shared='/share/LBI_share/public/retina/human\ snRNAseq\ data_10x_ChenLab_ZupanLab/humansn_10x_storage/00012_NeuNM_filtered_feature_bc_matrix'

# Normalise macaque data for logistic regression

In [3]:
file<-paste(data_path_ref,'passedQC.mtx',sep='')
matrix<-readMM(file)
file_prefix<-gsub("\\.mtx","",file)
row_names<-read.table(paste(file_prefix,'_genes.tsv',sep=''),sep='\t',header=TRUE)
col_data<-read.table(paste(file_prefix,'_cellData.tsv',sep=''),sep='\t',header=TRUE,row.names=1) 
rownames(matrix)<-row_names$rownames
colnames(matrix)<-rownames(col_data)
seurat.obj_ref<-CreateSeuratObject(counts=matrix, meta.data = col_data)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [4]:
# Normalise
seurat.obj_ref<-NormalizeData(seurat.obj_ref, normalization.method = "LogNormalize",scale.factor = 1000000)

In [5]:
# Keep variable genes intersecting human genes

In [6]:
seurat.obj_int<-readRDS(paste(data_path_ref,'scaled_integrated.rds',sep=''))

In [7]:
genes_ref_var=rownames(GetAssayData(object = seurat.obj_int, slot = "scale.data"))

In [8]:
human_dirs=list.dirs(path =data_path_shared_all, full.names = TRUE, recursive = FALSE)
genes_shared<-genes_ref_var
for (dir in human_dirs){
    genes_human=read.table(gzfile(paste(dir,'/features.tsv.gz',sep='')))$V2
    genes_shared<-intersect(genes_ref_var,genes_human)
    print(length(genes_shared))
}

[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203
[1] 2203


In [9]:
write.table(data.frame('genes'=genes_shared),file = paste(data_path,'genes_variable_human.tsv',sep=''), 
           row.names = FALSE,sep='\t')

In [10]:
normalised_ref<-GetAssayData(object = seurat.obj_ref, slot = 'data')
dim(normalised_ref)
normalised_ref<-normalised_ref[rownames(normalised_ref) %in% genes_shared,]
dim(normalised_ref)

[1]  27077 156644

[1]   2203 156644

In [32]:
file_norm=paste(data_path_ref,'normalisedlog.h5',sep='')
h5createFile(file_norm)
h5createGroup(file_norm,"ref")

file '/home/khrovatin/retinal/data/counts/normalisedlog.h5' already exists.



[1] FALSE

[1] TRUE

In [35]:
normalised_ref <- normalised_ref[ order(row.names(normalised_ref)), ]
h5write(as.matrix(normalised_ref), file_norm,"ref/matrix")
h5write(colnames(normalised_ref), file_norm,"ref/colnames")
h5write(rownames(normalised_ref), file_norm,"ref/rownames")
#writeMM(normalised_ref,paste(data_path_ref,'normalisedlog.mtx',sep=''))

You created a large dataset with compression and chunking.
The chunk size is equal to the dataset dimensions.
If you want to read subsets of the dataset, you should testsmaller chunk sizes to improve read times.



In [18]:
expression_matrix <- Read10X(data.dir = data_path_shared)

In [19]:
dim(expression_matrix)

[1] 32738 11245

In [20]:
col_counter<-colSums(expression_matrix!=0)
keep_cells<-col_counter>500
expression_matrix=expression_matrix[,keep_cells]

In [21]:
dim(expression_matrix)

[1] 32738 11194

In [22]:
seurat.obj_h_norm = CreateSeuratObject(counts = expression_matrix)
seurat.obj_h_norm<-NormalizeData(seurat.obj_h_norm, normalization.method = "LogNormalize",scale.factor = 1000000)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [23]:
normalised_h<-GetAssayData(object = seurat.obj_h_norm, slot = 'data')
dim(normalised_h)
normalised_h<-normalised_h[rownames(normalised_h) %in% genes_shared,]
dim(normalised_h)
#writeMM(normalised_h,paste(data_path,'00012_NeuNM_filtered_feature_bc_matrix-normalisedlog.mtx',sep=''))

[1] 32738 11194

[1]  2203 11194

In [36]:
h5createGroup(file_norm,"00012_NeuNM")
normalised_h <- normalised_h[ order(row.names(normalised_h)), ]
h5write(as.matrix(normalised_h), file_norm,"00012_NeuNM/matrix")
h5write(colnames(normalised_h), file_norm,"00012_NeuNM/colnames")
h5write(rownames(normalised_h), file_norm,"00012_NeuNM/rownames")

[1] TRUE

You created a large dataset with compression and chunking.
The chunk size is equal to the dataset dimensions.
If you want to read subsets of the dataset, you should testsmaller chunk sizes to improve read times.



# Integration of human to macaque

In [ ]:
genes_sample<-rownames(expression_matrix)
genes_shared<-intersect(genes_sample,genes_ref_var)

In [ ]:
length(genes_shared)

In [17]:
seurat.obj_h = CreateSeuratObject(counts = expression_matrix)

Warning message:
“Feature names cannot have underscores ('_'), replacing with dashes ('-')”


In [18]:
seurat.obj_h

An object of class Seurat 
32738 features across 11194 samples within 1 assay 
Active assay: RNA (32738 features)

In [11]:
seurat.obj_h<- SCTransform(seurat.obj_h, verbose = FALSE,return.only.var.genes = FALSE)

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

...

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):
“iteration limit reached”
Warning message in theta.ml(y = y, mu = fit$fitted):

In [12]:
seurat_both=list()
seurat_both[['ref']]<-seurat.obj_int
seurat_both[['sample']]<-seurat.obj_h

In [13]:
DefaultAssay(seurat_both[['ref']])<-'SCT'

In [14]:
genes_ref_var<-rownames(GetAssayData(object = seurat_both[['ref']], slot = "scale.data"))
genes_sample<-rownames(GetAssayData(object = seurat_both[['sample']], slot = "scale.data"))
genes_integration<-intersect(genes_sample,genes_ref_var)

In [16]:
length(genes_sample)
length(genes_integration)

[1] 23639

[1] 2025

In [16]:
seurat_both <- PrepSCTIntegration(object.list = seurat_both, anchor.features = genes_integration)

reference_dataset <- which(names(seurat_both) == "ref")

In [ ]:
anchors <- FindIntegrationAnchors(object.list = seurat_both, normalization.method = "SCT", 
    anchor.features = genes_integration, reference = reference_dataset)

Finding anchors between all query and reference datasets

Running CCA

Merging objects

Finding neighborhoods



In [ ]:
seurat_both1 <- IntegrateData(anchorset = anchors, normalization.method = "SCT")